<a href="https://colab.research.google.com/github/aditya248/Netflix-Exploratory-Data-Analysis/blob/main/BA775_TeamProject_B2T8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BA775 - Team Project - Team 8
##*Project Proposal*

###**Summary Research for Netflix on it’s library**

Our mission is to help Netflix summarize the details of their movie and TV show library to help them see trends on how their library has changed. We will also summarize Netflix materials based on popular genres and actors to compare with highest earning  and popular entertainment types so Netflix can maximize their user amount. 

Although not publicly avaliable, Netflix should use these summaries compared with the veiwership distrubtion in their library to ensure where their focus of content is (largest amounts) align with where their viewers are spending the most time watching those Movies and TV Shows.

**Source:**
https://www.kaggle.com/shivamb/netflix-sh

**Original Source:**
https://flixable.com/


In [ ]:
from google.colab import auth
auth.authenticate_user()

##**Part 1 -** Overview of Netflix Library
###This Section will provide  summary to Netflix of their  Library in order to understand the makeup.


**1. This Query will explore a count of the the Netflix library as well as content divide between movies and TV Shows** 

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT 
    
    COUNT(CASE WHEN type = 'Movie'
    THEN show_id END) AS movie_totals,
    COUNT(CASE WHEN type = 'TV Show' 
    THEN show_id END) AS tv_show_totals,
    COUNT(type) as total_library
 FROM `ba775-team8-b2.Netflix.netflix_titles` 

,movie_totals,tv_show_totals,total_library
0,5377,2410,7787


Of the almost 8000s unique videos on next more than 60% are movies. This shows netflix has favored movies over TV shows or they need more categories to breakdown what content types they have. 

**2. This query will summarize the dinstict type of ratings that exist  on Netflix  and breakdown the amount of content for each rating type on netflix.**

In [13]:
%%bigquery --project=ba775-team8-b2
SELECT rating as Type_Of_Rating, COUNT(*) as Total_Content 
FROM `ba775-team8-b2.Netflix.netflix_titles` 
WHERE rating IS NOT NULL
GROUP BY rating
ORDER BY Total_Content DESC
Limit 5
#Rating for both TV and movies

,Type_Of_Rating,Total_Content
0,TV-MA,2863
1,TV-14,1931
2,TV-PG,806
3,R,665
4,PG-13,386


Netlfix should compare their viewer count for shows in each rating type to compare if their viewers tend to perfer where their library is currently focused on ratings geared toward teenagres and adults. 

**3. This query will investigate which years do the most content that is added to the netflix library are released**

In [14]:
%%bigquery --project=ba775-team8-b2
SELECT release_year, COUNT(release_year) AS movie_added FROM `ba775-team8-b2.Netflix.netflix_titles`
GROUP BY release_year
ORDER BY movie_added DESC
Limit 5

,release_year,movie_added
0,2018,1121
1,2017,1012
2,2019,996
3,2016,882
4,2020,868


Focusing on the vast amount of content in recent years is good for young viewers, but could begin to focus on earlier years to attract a different subscriber type. 

**4. This query will breakdown for netflix which directors that are most used in their library.**

In [15]:
%%bigquery --project=ba775-team8-b2
SELECT director, COUNT(director) AS time_appeared
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE director IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
Limit 5

,director,time_appeared
0,"Raúl Campos, Jan Suter",18
1,Marcus Raboy,16
2,Jay Karas,14
3,Cathy Garcia-Molina,13
4,Jay Chapman,12


Having content from around 400 directors, netflix does not rely on too much. Their top uesed directly Jan suter Raul Campos only has about 2% of the library from his work. 

**5. This query will breakdown which countries Netflix rely on the most for production of content.**

In [16]:
%%bigquery --project=ba775-team8-b2
SELECT 
    country,
    COUNT(CASE WHEN type = 'Movie'
    THEN show_id END) AS movie_totals,
    COUNT(CASE WHEN type = 'TV Show' 
    THEN show_id END) AS tv_show_totals,
    COUNT(type) as total_content_prodcution
 FROM `ba775-team8-b2.Netflix.netflix_titles` 
 GROUP BY country
 ORDER BY total_content_prodcution DESC
 LIMIT 5;

,country,movie_totals,tv_show_totals,total_content_prodcution
0,United States,1850,705,2555
1,India,852,71,923
2,None,230,277,507
3,United Kingdom,193,204,397
4,Japan,69,157,226


The overhwhelming majority coming from the United States followed by India and the United Kingdom. The none catgeory should be organized to under where these 500 movies and TV shows are being made to accuratley understand what countries are msot utilized. 

##**Part 2 -** Netflix Movies
###This section will futher investigate the Movie library make up of Netflix. Combinig the data with the IMDB sites information on ratings and other popularity metrics will help Netflix know which movies to focus ono keeping and promoting on their main page

**6. The query will look at the average duration of movies.**

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT type,AVG(Updated_duration) AS avg_durationMovie FROM 
(SELECT type, CAST(REPLACE(duration,'min','') AS INT64) AS Updated_duration
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='Movie')
GROUP BY type

,type,avg_durationMovie
0,Movie,99.307978


**7. This query will investiage the highest rated movies on netflix page by using the popular IMDB ratings system**

In [19]:
%%bigquery --project=ba775-team8-b2
SELECT n.title, n.director, n.country, n.date_added, r.Rating
FROM `ba775-team8-b2.Netflix.netflix_titles` as n
INNER JOIN `ba775-team8-b2.Netflix.IMBd_movies_ratings` as r
   ON lower(n.title) = lower(r.title)
ORDER BY r.Rating DESC
LIMIT 10

,title,director,country,date_added,Rating
0,Inception,Christopher Nolan,"United States, United Kingdom","January 1, 2020",8.8
1,Dangal,Nitesh Tiwari,India,"June 21, 2017",8.8
2,The Intouchables,"Olivier Nakache, Éric Toledano",France,"January 11, 2021",8.6
3,Taare Zameen Par,Aamir Khan,India,"January 10, 2021",8.5
4,The Departed,Martin Scorsese,"United States, Hong Kong","January 1, 2021",8.5
5,The Lives of Others,Florian Henckel von Donnersmarck,Germany,"March 15, 2019",8.5
6,3 Idiots,Rajkumar Hirani,India,"August 1, 2019",8.4
7,Django Unchained,Quentin Tarantino,United States,"April 25, 2020",8.4
8,Inglourious Basterds,Quentin Tarantino,"Germany, United States","July 22, 2019",8.3
9,Paint It Black,Amber Tamblyn,United States,"February 1, 2018",8.3


Netflix should ensure these movies stay on their site for as much time as possilbe and regulary, promote and ensure users are aware that these movies exist in their library. 

**8.This query will again join with the IMDB data set to detial the top grossing box office movies that Netflix has in their library**

In [20]:
%%bigquery --project=ba775-team8-b2
SELECT n.title, n.director, n.country, n.date_added, r.Rating, r.Revenue__Millions_
FROM `ba775-team8-b2.Netflix.netflix_titles` as n
INNER JOIN `ba775-team8-b2.Netflix.IMBd_movies_ratings` as r
   ON lower(n.title) = lower(r.title)
ORDER BY r.Revenue__Millions_ DESC
Limit 5

,title,director,country,date_added,Rating,Revenue__Millions_
0,The Jungle Book,None,"India, Germany, France","May 11, 2019",7.5,364.00
1,Spider-Man 3,Sam Raimi,United States,"November 1, 2019",6.2,336.53
2,Indiana Jones and the Kingdom of the Crystal S...,Steven Spielberg,United States,"January 1, 2019",6.2,317.01
3,Inception,Christopher Nolan,"United States, United Kingdom","January 1, 2020",8.8,292.57
4,Star Trek,None,United States,"October 1, 2017",8.0,257.70


Movies that were extremely successfull like The Jungle Book and Spiderman 3 should be prioritized on the main page and to keep in the library. Additionally, netflix can study the types of moves that were successfull in order to narrow focus on what types of content to continue to add.

**9. This query will summarize the duration range of movies and the count of moveis that falls in each range.**

In [21]:
## Distribution of duration for Movies -Hanyu
%%bigquery --project=ba775-team8-b2
SELECT duration_range, COUNT(*) AS num_movies FROM 
(
SELECT CASE 
WHEN CAST(REPLACE(duration,'min','') AS INT64) <= 40 THEN '0 - 40'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 41 AND 80 THEN '41 - 80'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 81 AND 120 THEN '81 - 120'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 121 AND 160 THEN '121 - 160'
WHEN CAST(REPLACE(duration,'min','') AS INT64) BETWEEN 161 AND 200 THEN '161 - 200'
ELSE '201 +' END AS duration_range
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='Movie'
)
GROUP BY duration_range
ORDER BY CAST(SPLIT(duration_range,' ')[OFFSET(0)] as INT64)

,duration_range,num_movies
0,0 - 40,159
1,41 - 80,835
2,81 - 120,3372
3,121 - 160,874
4,161 - 200,122
5,201 +,15


Netflix can see that 1.5-2 hour movie duration contains  the vast majority of their movies library. 

##**Part 3 -** Netflix TV Shows
###This section will futher investigate the Tv Show libaray

**10. This query will look at average season of TV-shows.**

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT type, CAST(ROUND(AVG(Updated_duration)) AS INT64) AS avg_TV_Show_seasons FROM 
(SELECT type, CAST(REPLACE(REPLACE(duration,'s',''),'Seaon','') AS INT64) AS Updated_duration
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='TV Show')
GROUP BY type

,type,avg_TV_Show_seasons
0,TV Show,2


**11/12. The next  query will find what is the longest running Tv Show and its  length.**


In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT type,MAX(new_duration) AS MAX_TV_SHOW FROM
(SELECT type,CAST(REPLACE(REPLACE(REPLACE(duration,'min',''),'Season',''),'s','') AS INT64) AS new_duration
FROM `ba775-team8-b2.Netflix.netflix_titles`)
WHERE type = 'TV Show'
GROUP BY type

,type,MAX_TV_SHOW
0,TV Show,16


In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT show_id, title, type,duration FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type = 'TV Show'
      AND duration LIKE '16 S%'

,show_id,title,type,duration
0,s2539,Grey's Anatomy,TV Show,16 Seasons


Grey's Aanatotmy spanning 16 season is the longest running show on Netflix which should help them expect any show created or added is not likely to exceed this. 

**13. This query will isolate TV shows and breakdown the ratings by type.**

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT rating, COUNT(rating) AS count_rating FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='Movie' AND rating IS NOT NULL
GROUP BY rating
ORDER BY 2 DESC 

,rating,count_rating
0,TV-MA,1845
1,TV-14,1272
2,R,663
3,TV-PG,505
4,PG-13,386
5,PG,247
6,TV-Y,117
7,TV-G,111
8,TV-Y7,95
9,NR,79


Just like with Movies, the mature and tv-14 content holds the majority of the TV show library. Netflix should pair this with the distribution of views by rating to ensure that this divide in catelog matches their viewers. 

**14. In order for Netflix to understand how many season most TV shows have, This next ection will breakdown thoses statistiscs to help Netflix understand how much timer users will spend on a show**


In [24]:
%%bigquery --project=ba775-team8-b2
SELECT duration_season, COUNT(*) AS TV_Shows FROM 
(
SELECT CASE 
WHEN duration = '1 Season' THEN '1 Season'
WHEN duration = '2 Seasons' Then '2 Seasons'
WHEN duration = '3 Seasons' Then '3 Seasons'
WHEN duration = '4 Seasons' Then '4 Seasons'
WHEN duration = '5 Seasons' Then '5 Seasons'
WHEN duration = '6 Seasons' Then '6 Seasons'
WHEN duration = '7 Seasons' Then '7 Seasons'
WHEN duration = '8 Seasons' Then '8 Seasons'
WHEN duration = '9 Seasons' Then '9 Seasons'
WHEN duration = '10 Seasons' Then '10 Seasons'
WHEN duration = '11 Seasons' Then '11 Seasons'
WHEN duration = '12 Seasons' Then '12 Seasons'
ELSE '13 + Seasons' END AS duration_season
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type='TV Show'
)
GROUP BY duration_season
ORDER BY TV_Shows DESC
 


,duration_season,TV_Shows
0,1 Season,1608
1,2 Seasons,382
2,3 Seasons,184
3,4 Seasons,87
4,5 Seasons,58
5,6 Seasons,30
6,7 Seasons,19
7,8 Seasons,18
8,9 Seasons,8
9,10 Seasons,6


**15. This query will find the variance and standard deviation of TV show season**


In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT type, 
       AVG(new_duration * new_duration) - AVG(new_duration) * AVG(new_duration) AS Variance, 
       SQRT(AVG(new_duration*new_duration) - AVG(new_duration)*AVG(new_duration)) AS stdev FROM
(SELECT type,CAST(REPLACE(REPLACE(REPLACE(duration,'min',''),'Season',''),'s','') AS INT64) AS new_duration
FROM `ba775-team8-b2.Netflix.netflix_titles`)
WHERE type = 'TV Show'
GROUP BY type

,type,Variance,stdev
0,TV Show,2.547305,1.596028


##**Part 4 -** Trend over time
###This section will detail how Netflix library has trended in changing over time

**16. This will look at the trend of number of movies and shows that were addded to the netflix library by year**

In [25]:
%%bigquery --project=ba775-team8-b2
SELECT  RIGHT(date_added, 4) AS year_added,
    COUNT(release_year) as num_content_added
 FROM `ba775-team8-b2.Netflix.netflix_titles`
GROUP BY year_added
HAVING year_added <> 'none'
ORDER BY year_added DESC
Limit 5

,year_added,num_content_added
0,2021,117
1,2020,2009
2,2019,2153
3,2018,1685
4,2017,1225


Covid may have effected the amount of content that netflix could add last year, they must ensure they keep increase theri library to attract new customers and keep exisiting ones excited by new additions

**17. This query will output for Netflix the amount of content from each relese year based on rating type.**

In [26]:
%%bigquery --project=ba775-team8-b2
SELECT release_year, rating as Type_Of_Rating, COUNT(*) as Total_Content FROM `ba775-team8-b2.Netflix.netflix_titles` 
GROUP BY rating, release_year
ORDER BY release_year DESC
Limit 5

,release_year,Type_Of_Rating,Total_Content
0,2021,TV-14,7
1,2021,R,1
2,2021,TV-G,2
3,2021,TV-MA,14
4,2021,TV-Y,2


This could help them find gaps in their library to add G or PG rated shows from early years that may be wanted by older users to show their younger families. 

**18. Bulding off the last query, this one will find the by year, which rating type has the largest amount of content on the Netflix library**

In [27]:
%%bigquery --project=ba775-team8-b2
SELECT tb1.release_year, tb1.Max_Content_Count, tb2.Type_Of_Rating 
FROM
(
    SELECT release_year, MAX(Total_Content) as Max_Content_Count 
    FROM (SELECT release_year, rating as Type_Of_Rating, COUNT(*) as Total_Content FROM `ba775-team8-b2.Netflix.netflix_titles` GROUP BY rating, release_year)
    GROUP BY release_year
) as tb1
INNER JOIN (SELECT release_year, rating as Type_Of_Rating, COUNT(*) as Total_Content FROM `ba775-team8-b2.Netflix.netflix_titles` GROUP BY rating, release_year) as tb2
ON tb1.release_year = tb2.release_year
WHERE tb1.Max_Content_Count = tb2.Total_Content
ORDER BY tb1.release_year DESC
Limit 5

,release_year,Max_Content_Count,Type_Of_Rating
0,2021,14,TV-MA
1,2020,427,TV-MA
2,2019,499,TV-MA
3,2018,544,TV-MA
4,2017,453,TV-MA


**19. This query will breakdown which months in a given year over the duration of Netflix existence do they add the most new content to the library**



In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT month_added, COUNT(*) AS num_content_added FROM 
(
SELECT CASE 
WHEN date_added LIKE 'January%' THEN 'JAN'
WHEN date_added LIKE 'February%' THEN 'FEB'
WHEN date_added LIKE 'March%' THEN 'MAR'
WHEN date_added LIKE 'April%' THEN 'APR'
WHEN date_added LIKE 'May%' THEN 'MAY'
WHEN date_added LIKE 'June%' THEN 'JUN'
WHEN date_added LIKE 'July%' THEN 'JUL'
WHEN date_added LIKE 'August%' THEN 'AUG'
WHEN date_added LIKE 'September%' THEN 'SEP'
WHEN date_added LIKE 'October%' THEN 'OCT'
WHEN date_added LIKE 'November%' THEN 'NOV'
WHEN date_added LIKE 'December%' THEN 'DEC'
END AS month_added
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE date_added IS NOT NULL 
)
GROUP BY month_added
HAVING month_added IS NOT NULL 
ORDER BY num_content_added DESC


,month_added,num_content_added
0,DEC,817
1,OCT,780
2,JAN,746
3,NOV,730
4,MAR,661
5,SEP,614
6,AUG,612
7,APR,596
8,JUL,592
9,JUN,538


This trend of of the winter and fall months having the most amount of content added to Netflix should be compared with usage frequencies and new subrcribtions added to help Netflix maximize windows of attracting new users and keeping the base they have.

##**Part 5 -** Maximizing User Amount / Research Analysis Conclusion
###The United States users make up roughly 1/3rd of the total users in the Netflix library. As shown above how they majority of their library makes up content from the U.S. as well they should find ways to maximize their users in this country

**20. Recent research has shown that PG-13 movies are the most profitable rating type in theatres due to their broad audiecne reach. This query will invesitage what percent of movies produced in the U.S. have this rating (https://stephenfollows.com/which-mpaa-rating-earns-the-most-money/)**

In [ ]:
%%bigquery --project=ba775-team8-b2

Select ((SELECT COUNT(*) FROM `ba775-team8-b2.Netflix.netflix_titles` WHERE country = 'United States' and  type = 'Movie' AND rating = 'PG-13')*100/COUNT(*)) as US_PG13_movies_percentage
FROM `ba775-team8-b2.Netflix.netflix_titles` 
WHERE country = 'United States' and  type = 'Movie'

,US_PG13_movies_percentage
0,12.27027


Netflix may consider increasing the amount of moveis that are PG-13 from U.S. productions to further reach the broad audience it captures in theatres. 

**21. This study has found over the last 25 years the ranking of highest grossing movie genres in North America. This query will investigate where the top 5 genres rank in amount of movies prodcued in the U.S. in the netflix library**

In [ ]:
%%bigquery --project=ba775-team8-b2
SELECT Top_grossing_movie_genres, COUNT(*) AS movie_genre_breakdown,  
FROM 
(
SELECT 
CASE 
  WHEN listed_in LIKE '%Adventure%' OR  listed_in LIKE '%Action%' THEN 'Rank 1: Adventure/Action'
  WHEN listed_in LIKE '%Drama%' THEN 'Rank 2: Drama'
  WHEN listed_in LIKE '%Comedy%' or listed_in LIKE '%Comedies%'  THEN 'Rank 3: Comedy'
  WHEN listed_in LIKE '%Horror%' THEN 'Rank 4: Horror'
  WHEN listed_in LIKE '%Thriller%' THEN 'Rank 5: Thriller'
  WHEN listed_in LIKE '%Sci-Fi%' THEN 'Rank 6: Sci-fi'
  WHEN listed_in LIKE '%Document%' THEN 'Rank 7: Documentaries'
  WHEN listed_in LIKE '%Musical%' THEN 'Rank 8: Musical'
  ELSE 'Other Genres' END AS Top_grossing_movie_genres
FROM `ba775-team8-b2.Netflix.netflix_titles`
WHERE type = 'Movie'
AND country IN ('United States', 'Canada', 'Mexico')
AND listed_in IS NOT NULL
)
GROUP BY Top_grossing_movie_genres
ORDER BY movie_genre_breakdown DESC  

,Top_grossing_movie_genres,movie_genre_breakdown
0,Rank 3: Comedy,580
1,Rank 2: Drama,528
2,Rank 7: Documentaries,406
3,Rank 1: Adventure/Action,210
4,Other Genres,127
5,Rank 4: Horror,111
6,Rank 5: Thriller,52
7,Rank 8: Musical,11
8,Rank 6: Sci-fi,8


Action (47 billion) and Adventure(64 billion) rank highest in the charts at roughly 111 billion dollars in revenue, and Drama is section in 35 billion. Netflix should promote these on their site more and  add more content in these genres in order to increase users that are flocking to the theatres to see these movies over the last two decades.